In [1]:
root_path = "/tf/home/sergio/Tesis"

In [2]:
import tensorflow as tf
from tensorflow.python.compiler.tensorrt import trt_convert as trt
import numpy as np
from tensorflow.compat.v1 import InteractiveSession

In [3]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [4]:
import os

In [5]:
os.getcwd()

'/tf/home/sergio/Tesis/TinyYOLOv3-Pedestrian-Detection/Deployment'

In [6]:
os.listdir()

['Yolo_Functional_to_TensorRT.ipynb',
 'onnx2trt.py',
 'saved_model_production',
 'engine.py',
 'TF-TRT to UFF-TRT.ipynb',
 'saved_model_tensorrt_vectorized',
 'pedestrians_example.jpg',
 'weights_functional_one_class',
 '.ipynb_checkpoints',
 'saved_model_vectorized',
 'Save the model.ipynb',
 'Get the ONNX model.ipynb',
 'saved_model_tensorrt_production_with_NMS',
 'test.py',
 'yolov3-tiny.weights',
 'converted_model.tflite',
 'pedestrians.jpg',
 'Testing_model in CPU.ipynb',
 'saved_model_with_NMS',
 'inference.py',
 'saved_model_functional_production_with_NMS',
 '__pycache__',
 'yolov3-tiny.cfg',
 'Get the TensorRT model.ipynb',
 'saved_model',
 'Testing_functional_model_darknet_weights.ipynb',
 'weights_fine_tuning_functional',
 'saved_model_tensorrt_production',
 'yolo_pedestrian.plan',
 'model.onnx',
 'saved_model_functional_tensorrt_production_with_NMS',
 'saved_model_production_with_NMS',
 'Tiny_YOLOv3_Functional.ipynb',
 'saved_model_tensorrt',
 'saved_model_tensorrt_with_NMS

In [7]:
!mkdir -p 'saved_model_functional_tensorrt_production_with_NMS/my_model'

In [8]:
conversion_params = trt.DEFAULT_TRT_CONVERSION_PARAMS
conversion_params = conversion_params._replace(max_workspace_size_bytes=(1<<32))
conversion_params = conversion_params._replace(precision_mode="FP16")
conversion_params = conversion_params._replace(maximum_cached_engines=100)

def my_input_fn():
    for _ in range(1):
        inp1 = tf.random.normal(shape=(1, 416, 416, 3),dtype=tf.dtypes.float32)#.astype(tf.float32)
        #inp2 = np.random.normal(size=(1, 416, 416, 3)).astype(np.float32)
        yield (inp1,)

def getTrtGraphDef(tf_version):
    if tf_version == 2:
        converter = trt.TrtGraphConverterV2(
            input_saved_model_dir='saved_model_functional_production_with_NMS/my_model',
            conversion_params=conversion_params)
        converter.convert()
        converter.build(input_fn=my_input_fn)
        converter.save('saved_model_functional_tensorrt_production_with_NMS/my_model')
        
        saved_model_loaded = tf.saved_model.load('saved_model_functional_tensorrt_production_with_NMS/my_model')
        graph_func = saved_model_loaded.signatures["serving_default"]
        return graph_func.graph.as_graph_def()
    elif tf_version == 1:
        # only works if is_dynamic_op=False and precision_mode=”FP32” or “FP16”.
        return create_inference_graph(...)

trt_graph = getTrtGraphDef(2)

INFO:tensorflow:Linked TensorRT version: (6, 0, 1)
INFO:tensorflow:Loaded TensorRT version: (6, 0, 1)
INFO:tensorflow:Assets written to: saved_model_functional_tensorrt_production_with_NMS/my_model/assets


In [12]:
for n in trt_graph.node:
    print(n)
    if n.op == "TRTEngineOp":
        print("Node: %s, %s" % (n.op, n.name.replace("/", "_")))
        with tf.gfile.GFile("%s.plan" % (n.name.replace("/", "_")), 'wb') as f:
            f.write(n.attr["serialized_segment"].s)
    else:
        print("Exclude Node: %s, %s" % (n.op, n.name.replace("/", "_")))

name: "input_1"
op: "Placeholder"
attr {
  key: "_user_specified_name"
  value {
    s: "input_1"
  }
}
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "shape"
  value {
    shape {
      dim {
        size: -1
      }
      dim {
        size: 416
      }
      dim {
        size: 416
      }
      dim {
        size: 3
      }
    }
  }
}

Exclude Node: Placeholder, input_1
name: "PartitionedCall"
op: "PartitionedCall"
input: "input_1"
attr {
  key: "Tin"
  value {
    list {
      type: DT_FLOAT
    }
  }
}
attr {
  key: "Tout"
  value {
    list {
      type: DT_FLOAT
    }
  }
}
attr {
  key: "_read_only_resource_inputs"
  value {
    list {
    }
  }
}
attr {
  key: "config"
  value {
    s: ""
  }
}
attr {
  key: "config_proto"
  value {
    s: "\n\007\n\003CPU\020\001\n\007\n\003GPU\020\0012\007 \001*\0010J\0008\001\202\001\000"
  }
}
attr {
  key: "executor_type"
  value {
    s: ""
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_pr